In [1]:
import torch
import torch.nn.functional as F
import math
print("before tex import")
import transformer_engine as te
import transformer_engine_torch as tex

before tex import


In [2]:
from transformer_engine.pytorch.attention import apply_rotary_pos_emb

In [3]:
def compare_rope_outputs(t, freqs_s11d, freqs_sb1d):
    output1 = tex.fused_rope_forward(t, freqs_s11d, torch.Tensor(), False)
    output2 = tex.fused_rope_forward(t, freqs_sb1d, torch.Tensor(), False)
    print(output1, output2, sep="\n")
    assert torch.allclose(output1, output2)
    return output1, output2

In [4]:
torch.manual_seed(0)
b = 2
s = 3
h = 2
d = 4

In [5]:
freqs_s11d = torch.ones(s, 1, 1, d).cuda() * math.pi/4
freqs_sb1d = freqs_s11d.broadcast_to(s, b, 1, d).clone()
t = torch.ones(s, b, h, d).cuda()

print(freqs_s11d.shape, freqs_sb1d.shape)

torch.Size([3, 1, 1, 4]) torch.Size([3, 2, 1, 4])


In [6]:
freqs_s11d

tensor([[[[0.7854, 0.7854, 0.7854, 0.7854]]],


        [[[0.7854, 0.7854, 0.7854, 0.7854]]],


        [[[0.7854, 0.7854, 0.7854, 0.7854]]]], device='cuda:0')

In [7]:
freqs_s11d.stride()

(4, 4, 4, 1)

In [8]:
output = tex.fused_rope_forward(t, freqs_s11d, torch.Tensor(), False)

4, 0
4, 4, 4, 1, 
nvt_fused_rope_fwd: 4, 0fused_rope_fwd: 4, 0fused_rope_fwd_launcher: 4, 0thread_id: 0, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 1, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 2, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 3, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 0, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 1, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 2, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 3, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin:

In [9]:
output_unfused=apply_rotary_pos_emb(
    t,
    freqs_s11d,
    tensor_format="sbhd",
    fused=False,
)

In [10]:
output_unfused

tensor([[[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]],


        [[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]],


        [[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]]],
       device='cuda:0')

In [11]:
output1, output2 = compare_rope_outputs(t, freqs_s11d, freqs_sb1d)

4, 0
4, 4, 4, 1, 
nvt_fused_rope_fwd: 4, 0fused_rope_fwd: 4, 0fused_rope_fwd_launcher: 4, 08, 4
8, 4, 4, 1, 
nvt_fused_rope_fwd: 8, 4fused_rope_fwd: 8, 4fused_rope_fwd_launcher: 8, 4thread_id: 0, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 1, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 2, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 3, s_id: 0, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 0, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 1, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107
thread_id: 2, s_id: 1, b_id: 0, freq_stride_s: 4, freq_stride_b: 0, freq: 0.785398, v_sin: 0.707107, v_cos: 0.707107

In [16]:
freqs_s11d = torch.randn(s, 1, 1, d).cuda()
freqs_sb1d = freqs_s11d.broadcast_to(s, b, 1, d).clone()

print(freqs_s11d.shape, freqs_sb1d.shape)

torch.Size([6, 1, 1, 4]) torch.Size([6, 2, 1, 4])


In [9]:
output1, output2 = compare_rope_outputs(t, freqs_s11d, freqs_sb1d)

tensor([[[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]],


        [[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]],


        [[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]]],


        [[[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00],
          [-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.4142e+00]],

         [[-5.9605e-08, -5.9605e-08,  1.4142e+00,  1.0000e+00],
          [-5.9605e-

AssertionError: 

In [ ]:
freqs_s11d = torch.randn(s, 1, 1, d).cuda()
print(freqs_s11d)
freqs_sb1d = freqs_s11d.broadcast_to(s, b, 1, d).clone()
print(freqs_sb1d)
assert torch.all(torch.eq(freqs_sb1d[:, 0, ...], freqs_sb1d[:, 1, ...]))

comp

before tex import
